In [1]:
import pandas as pd
from apyori import apriori

df = pd.read_csv('Groceries_dataset.csv')
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [2]:
member_shopping_frequency = df.groupby('Member_number')['Date'].count().sort_values(ascending=False)
print(member_shopping_frequency)

Member_number
3180    36
3737    33
3050    33
2051    33
2271    31
        ..
4451     2
3990     2
2213     2
2852     2
1368     2
Name: Date, Length: 3898, dtype: int64


In [3]:
group_same_customer = df.sort_values(by = 'Member_number',ascending = True)
group_same_customer['itemDescription'].str.strip()

1629                    soda
13331             whole milk
8395              whole milk
4843                 sausage
17778     pickled vegetables
                ...         
34885    semi-finished bread
25489       other vegetables
9340            bottled beer
27877                 onions
3578                    soda
Name: itemDescription, Length: 38765, dtype: object

In [4]:
transactions_list = []
for _, group in group_same_customer.groupby('Member_number'):
    customer_purchases = group['itemDescription'].tolist()
    transactions_list.append(customer_purchases)

In [5]:
association_rules = apriori(transactions_list, min_support=0.001, min_confidence=0.05, min_lift=4, min_length=2,max_length = 2)
rules_list = list(association_rules)
rules_list

[RelationRecord(items=frozenset({'soups', 'Instant food products'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Instant food products'}), items_add=frozenset({'soups'}), confidence=0.06666666666666667, lift=5.413888888888889), OrderedStatistic(items_base=frozenset({'soups'}), items_add=frozenset({'Instant food products'}), confidence=0.08333333333333333, lift=5.413888888888889)]),
 RelationRecord(items=frozenset({'soft cheese', 'canned fruit'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'canned fruit'}), items_add=frozenset({'soft cheese'}), confidence=0.19047619047619047, lift=5.050858438613541)]),
 RelationRecord(items=frozenset({'canned vegetables', 'rum'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'rum'}), items_add=frozenset({'canned vegetables'}), confidence=0.125, lift=6.090625)]),
 RelationRecord(items=frozenset({'tea', 'cat food'}), supp

In [6]:
def apriori_df(results):
    extracted_data = []

    for result in results:
        items_base = tuple(result.ordered_statistics[0].items_base)[0]
        items_add = tuple(result.ordered_statistics[0].items_add)[0]
        support = result.support
        confidence = result.ordered_statistics[0].confidence
        lift = result.ordered_statistics[0].lift

        extracted_data.append((items_base, items_add, support, confidence, lift))

    return extracted_data

resultsinDataFrame = pd.DataFrame(apriori_df(rules_list), columns = ['antecedent', 'consequent', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame[:10]

,antecedent,consequent,Support,Confidence,Lift
0,Instant food products,soups,0.001026,0.066667,5.413889
1,canned fruit,soft cheese,0.001026,0.190476,5.050858
2,rum,canned vegetables,0.001026,0.125000,6.090625
3,tea,cat food,0.001796,0.259259,5.909898
4,ready soups,chewing gum,0.001026,0.266667,5.973946
5,cookware,ice cream,0.001026,0.235294,4.168984
6,curd cheese,mustard,0.001283,0.108696,4.655996
7,decalcifier,dessert,0.001026,0.444444,5.140785
8,dental care,seasonal products,0.001026,0.121212,4.678068
9,dog food,frozen potato products,0.001283,0.074627,4.040216
